# Import Lib

In [1]:
import open3d as o3d
import numpy as np

import pandas
from sklearn.decomposition import PCA

import math
import copy

In [2]:
# add aixs
aix_points = [[0, 0, 0],
              [0, 0, 100],
              [400, 0, 0],
              [0, 100, 0],
              [-400, 0, 0],]
aix_lines = [[0, 1], # z-aix
             [0, 2], # x-aix
             [0, 3]] # y-aix

colors = [[1,0,1], [0,0,0], [0,0,0]]
aix_line_set = o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(aix_points),
                                    lines=o3d.utility.Vector2iVector(aix_lines))
aix_line_set.colors = o3d.utility.Vector3dVector(colors)

# Read scan model
## import obj file

In [3]:
scan_obj = o3d.io.read_triangle_mesh("./data/femur_half_4.obj")
print(scan_obj)
o3d.visualization.draw_geometries([scan_obj], mesh_show_wireframe=True)

geometry::TriangleMesh with 35027 points and 68786 triangles.


## Scale unit length to 1mm (coordinate 1000x)

In [4]:
points_center = scan_obj.get_center()
print(points_center)
scan_obj.scale(1000.0, points_center)

[ 0.24559977 -0.49631218 -0.23152044]


geometry::TriangleMesh with 35027 points and 68786 triangles.

## change to point cloud

In [5]:
number_of_points = np.asarray(scan_obj.vertices).shape[0]
scan_obj.compute_vertex_normals()
scan_pcd = scan_obj.sample_points_uniformly(number_of_points)

In [6]:
center = scan_pcd.get_center()
center

array([ 0.4843591 , -2.06221251, -4.96408358])

# --------------------------------------

## delete plane
### find plane using RANSAC: plane function: ax + by + cz + d = 0

In [31]:
plane_model, inliers = scan_pcd.segment_plane(distance_threshold=2,
                                              ransac_n=3,
                                              num_iterations=1000)
[a, b, c, d] = plane_model
plane = plane_model
print(f"Plane equation: {a:.5f}x + {b:.5f}y + {c:.5f}z + {d:.5f} = 0")

# floor
inlier_cloud = scan_pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

# bone 
bone_cloud = scan_pcd.select_by_index(inliers, invert=True)

Plane equation: 0.00295x + 0.99997y + -0.00717z + 6.81117 = 0


## Delete outliers

In [32]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [33]:
cl, ind = bone_cloud.remove_statistical_outlier(nb_neighbors=10,
                                                std_ratio=2.0)
# display outliers
display_inlier_outlier(bone_cloud, ind)

Showing outliers (red) and inliers (gray): 


In [34]:
bone_cloud = bone_cloud.select_by_index(ind)

### project point to plane 
link:  \
https://math.stackexchange.com/questions/100761/how-do-i-find-the-projection-of-a-point-onto-a-plane


In [35]:
def point_to_plane(point, plane):
    A = plane[0]
    B = plane[1]
    C = plane[2]
    D = plane[3]

    xi = point[0]
    yi = point[1]
    zi = point[2]

    t = (A*xi + B*yi + C*zi + D) / (A**2 + B**2 + C**2)
    x = xi - A*t
    y = yi - B*t
    z = zi - C*t
    
    res = [x, y, z]
    return res

In [36]:
bone_points = np.array(bone_cloud.points)
bone_points

array([[ 192.00781961,   27.11725688,  121.8752335 ],
       [ 194.76741818,   26.61129308,  122.60028947],
       [ 198.92217008,   27.38880825,  122.01176158],
       ...,
       [-158.98695728,    1.04439092,    9.68335274],
       [-156.82001202,    0.77605994,    9.99689326],
       [-158.83272677,    1.6296953 ,    7.47385414]])

In [40]:
proj_bone_points = []
for i in range(bone_points.shape[0]):
    point = bone_points[i]
    proj_bone_points.append(point_to_plane(point, plane))

bone_points = np.array(proj_bone_points)
bone_points

array([[ 191.90877717,   -6.50121047,  122.11626931],
       [ 194.66985762,   -6.50417221,  122.83771889],
       [ 198.82227055,   -6.5205843 ,  122.25488326],
       ...,
       [-159.0085152 ,   -6.27312185,    9.73581743],
       [-156.84079166,   -6.2772737 ,   10.04746386],
       [-158.85605695,   -6.28938162,    7.5306319 ]])

In [42]:
bone_pcd = o3d.geometry.PointCloud()
bone_pcd.points = o3d.utility.Vector3dVector(bone_points)

o3d.visualization.draw_geometries([bone_pcd, aix_line_set])

In [ ]:
bone_points = bone_points - bone_points.mean(axis=0, keepdims=True)

## save file